In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df=pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")
df1=pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tweet_id                10980 non-null  int64 
 1   airline_sentiment       10980 non-null  object
 2   airline                 10980 non-null  object
 3   airline_sentiment_gold  31 non-null     object
 4   name                    10980 non-null  object
 5   negativereason_gold     24 non-null     object
 6   retweet_count           10980 non-null  int64 
 7   text                    10980 non-null  object
 8   tweet_coord             776 non-null    object
 9   tweet_created           10980 non-null  object
 10  tweet_location          7430 non-null   object
 11  user_timezone           7403 non-null   object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB
None


In [7]:
print(df.airline_sentiment.value_counts())

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64


In [8]:
df.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)
df1.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)

In [17]:
df.head()

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...


## DATA CLEANING

In [9]:
from  nltk.tokenize  import WordPunctTokenizer
tok = WordPunctTokenizer()

In [10]:
import re
from bs4 import BeautifulSoup

In [11]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [20]:
Training=df.text
Testing=df1.text

In [21]:
Train_result=[]
Test_result=[]
for i in Training:
    Train_result.append((cleaner(i)))
for i in Testing:
    Test_result.append((cleaner(i)))

## Testing Score

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## Vectorizer

In [23]:
Tfid_Vectorizer=TfidfVectorizer(ngram_range=(1,2),max_features=2000)
X_train_feature=Tfid_Vectorizer.fit_transform(Train_result)
X_test_feature=Tfid_Vectorizer.transform(Test_result)

In [24]:
print(X_train_feature.shape)
print(X_test_feature.shape)

(10980, 2000)
(3660, 2000)


## Logistic


In [57]:
from sklearn.linear_model import LogisticRegression
abc=LogisticRegression(C=4)


In [58]:
abc

LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
abc.fit(X_train_feature,df.airline_sentiment)

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
Sentiment1=abc.predict(X_test_feature)

In [61]:
np.savetxt('SentimentSVM2.csv',Sentiment1,fmt="%s")

In [62]:
abc.score(X_train_feature,df.airline_sentiment)

0.8815118397085611